<a href="https://colab.research.google.com/github/Jathu03/Assignment-02/blob/main/Assignment_2_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def pairwise_squared_euclidean_distance(X, Y):
    # ||x_i - y_j||^2 = ||x_i||^2 + ||y_j||^2 - 2 * <x_i,y_j>
    # Computing the squared norms of matrices X and Y
    X_norm_squared = np.sum(X**2, axis=1).reshape(-1, 1)  # getting norm of X as a column vector of shape =(m, 1)
    Y_norm_squared = np.sum(Y**2, axis=1).reshape(1, -1)  # getting norm of Y as a row vector of shape = (1, n)

    # Computing the dot product of X and Y
    X_Y_dot_product = np.dot(X, Y.T)

    return X_norm_squared + Y_norm_squared - 2 * X_Y_dot_product


X = np.random.randn(5, 3)  # 5 x 3 matrix
Y = np.random.randn(6, 3)  # 6 x 3 matrix

Z = pairwise_squared_euclidean_distance(X, Y)

print(Z)

[[16.98566131  3.20993322 10.70589783  2.60898703  4.76761628  1.37645994]
 [ 8.22973974  4.83917687 15.98532335  9.01443039  3.9736362   1.89733064]
 [ 8.42291213  0.33576711  3.53614334  2.12159843  1.39766252  1.53552856]
 [ 2.32917512  5.51177081  4.29448351  4.68054086  1.23135024  5.70316062]
 [13.30410114  1.46427728  2.32975776  3.1703183   4.39376562  4.69652359]]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import accuracy_score
from collections import Counter
import numpy as np

In [ ]:
# Code to load the dataset and extract feature embedding

# Define the transformation to be applied to the images
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset from Google Drive
dataset = ImageFolder(root='/content/drive/MyDrive/caltech101', transform=data_transform)

# Split dataset randomly into training and testing
train_size = int(2/3 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Load Pre-trained ResNet-50 Model
resnet50 = models.resnet50(pretrained=True)
resnet50 = torch.nn.Sequential(*list(resnet50.children())[:-1])  # Remove the last classification layer
resnet50.eval()  # Set model to evaluation mode

# Extract the feature embeddings
def extract_features(data_loader, model):
    features = []
    labels = []

    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs = model(inputs)
            outputs = outputs.view(outputs.size(0), -1)  # Flatten the output tensor
            features.append(outputs.numpy())
            labels.append(targets.numpy())

    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

# Extract features from train and test datasets
train_features, train_labels = extract_features(train_loader, resnet50)
test_features, test_labels = extract_features(test_loader, resnet50)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 196MB/s]


In [ ]:
#  k-NN Classification (Question 3.a)
def knn_predict(test_features, train_features, train_labels, k):
    predictions = []

    # Calculate the euclidean distances
    distances = pairwise_squared_euclidean_distance(test_features, train_features)

    # Iterate over each test point
    for i in range(test_features.shape[0]):
        # Get the indices of the k nearest neighbors (smallest distances)
        k_nearest_indices = np.argsort(distances[i])[:k]

        # Get the labels of the k nearest neighbors
        k_nearest_labels = [train_labels[j] for j in k_nearest_indices]

        # Perform majority vote
        most_common_label = Counter(k_nearest_labels).most_common(1)[0][0]
        predictions.append(most_common_label)

    return np.array(predictions)

# k-NN Classification
k = 5

# Evaluate on Test Set
test_predictions = knn_predict(test_features, train_features, train_labels, k)
accuracy = accuracy_score(test_labels, test_predictions)

print(f"Test Set Accuracy: {accuracy * 100:.2f}%")

Test Set Accuracy: 88.22%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Train Linear Classifier (Question 3.b)

# Convert features and labels into tensors
train_features_tensor = torch.tensor(train_features, dtype=torch.float32)  # Removed .to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)  # Removed .to(device)

# Define the linear classifier
num_classes = len(dataset.classes)
input_dim = train_features.shape[1]

classifier = nn.Linear(input_dim, num_classes)  # Removed .to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Training for the linear classifier
epochs = 20
batch_size = 32
num_samples = train_features_tensor.shape[0]

for epoch in range(epochs):
    classifier.train()
    total_loss = 0

    # Mini-batch training
    permutation = torch.randperm(num_samples)
    for i in range(0, num_samples, batch_size):
        indices = permutation[i:i + batch_size]
        batch_features = train_features_tensor[indices]
        batch_labels = train_labels_tensor[indices]

        # Forward pass
        outputs = classifier(batch_features)
        loss = criterion(outputs, batch_labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")

# Evaluate the Test Set
classifier.eval()
test_features_tensor = torch.tensor(test_features, dtype=torch.float32)  # Removed .to(device)

with torch.no_grad():
    test_outputs = classifier(test_features_tensor)
    _, test_predictions = torch.max(test_outputs, 1)

test_accuracy = accuracy_score(test_labels, test_predictions.numpy())  # No need for .cpu()
print(f"Test Set Accuracy: {test_accuracy * 100:.2f}%")


Epoch [1/20], Loss: 244.8940
Epoch [2/20], Loss: 51.4168
Epoch [3/20], Loss: 30.3171
Epoch [4/20], Loss: 19.9645
Epoch [5/20], Loss: 13.6421
Epoch [6/20], Loss: 10.7997
Epoch [7/20], Loss: 7.8402
Epoch [8/20], Loss: 6.6773
Epoch [9/20], Loss: 5.0535
Epoch [10/20], Loss: 4.2201
Epoch [11/20], Loss: 3.4698
Epoch [12/20], Loss: 2.9287
Epoch [13/20], Loss: 2.6041
Epoch [14/20], Loss: 2.5710
Epoch [15/20], Loss: 2.1450
Epoch [16/20], Loss: 1.7471
Epoch [17/20], Loss: 1.4299
Epoch [18/20], Loss: 1.3995
Epoch [19/20], Loss: 1.2267
Epoch [20/20], Loss: 1.0437
Test Set Accuracy: 92.88%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Load and Split the Caltech-101 Dataset with Data Augmentation
data_transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

data_transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the dataset from the drive and split it into training and testing
dataset = ImageFolder(root='/content/drive/MyDrive/caltech101', transform=data_transform_train)
train_size = int(2 / 3 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Apply test transform to the test dataset
test_dataset.dataset.transform = data_transform_test

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Load Pre-trained ResNet-50 Model and Modify for Fine-tuning
resnet50 = models.resnet50(pretrained=True)

# Modify the fully connected layer to match the number of classes in Caltech-101
num_classes = len(dataset.classes)
resnet50.fc = nn.Linear(resnet50.fc.in_features, num_classes)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.parameters(), lr=0.0001)  # Use a small learning rate for fine-tuning
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  # Learning rate scheduler

# Train the Network
epochs = 10
for epoch in range(epochs):
    resnet50.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        # Forward pass
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    scheduler.step()  # Adjust the learning rate
    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")

# Evaluate on the test Set
resnet50.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = resnet50(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Set Accuracy: {test_accuracy:.2f}%")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 296.9842, Train Accuracy: 68.96%
Epoch [2/10], Loss: 43.8542, Train Accuracy: 95.62%
Epoch [3/10], Loss: 15.2190, Train Accuracy: 98.59%
Epoch [4/10], Loss: 5.9009, Train Accuracy: 99.46%
Epoch [5/10], Loss: 6.9574, Train Accuracy: 99.38%
Epoch [6/10], Loss: 7.3933, Train Accuracy: 99.28%
Epoch [7/10], Loss: 14.3735, Train Accuracy: 98.29%
Epoch [8/10], Loss: 7.0534, Train Accuracy: 99.31%
Epoch [9/10], Loss: 2.5168, Train Accuracy: 99.85%
Epoch [10/10], Loss: 1.6772, Train Accuracy: 99.92%
Test Set Accuracy: 95.05%
